In [26]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D,ConvLSTM2D

from tensorflow.keras.backend import argmax
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.utils import to_categorical

In [27]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataY.append(dataset[i + look_back, 0]) 
        dataX.append(a)
    return numpy.array(dataX), numpy.array(dataY)

In [28]:
def create_resultset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        dataY.append(dataset.iloc[i + look_back, 0]) 
    return  numpy.array(dataY)

In [29]:
import datasets 
from datasets import load_ypred, load_usps, load_mrec

Y=load_ypred()

#Y.shape
Ysize=len(numpy.bincount(Y.iloc[:,0]))

MM (4030, 1)
YPRED: (4030, 1)


In [30]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 0.999))
YM = scaler.fit_transform(Y)
#YM=Y.loc[:,:]
#YM

numpy.bincount(Y.iloc[:,0])

array([219, 491, 466, 208, 378, 268, 298,  82,  64, 448, 128, 240, 152,
       266, 196, 126])

In [31]:
look_back = 20
trainX, _ = create_dataset(YM, look_back)
trainY =create_resultset(Y,look_back)
Ysize=16


In [32]:
trainY[1:6]
trainYO = to_categorical(trainY, num_classes=Ysize)
trainYO

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [33]:
trainX[1:6]

array([[0.5994, 0.1332, 0.7992, 0.3996, 0.8658, 0.5994, 0.1332, 0.7992,
        0.5994, 0.1332, 0.7992, 0.3996, 0.8658, 0.5994, 0.1332, 0.7992,
        0.5994, 0.1332, 0.666 , 0.999 ],
       [0.1332, 0.7992, 0.3996, 0.8658, 0.5994, 0.1332, 0.7992, 0.5994,
        0.1332, 0.7992, 0.3996, 0.8658, 0.5994, 0.1332, 0.7992, 0.5994,
        0.1332, 0.666 , 0.999 , 0.8658],
       [0.7992, 0.3996, 0.8658, 0.5994, 0.1332, 0.7992, 0.5994, 0.1332,
        0.7992, 0.3996, 0.8658, 0.5994, 0.1332, 0.7992, 0.5994, 0.1332,
        0.666 , 0.999 , 0.8658, 0.5994],
       [0.3996, 0.8658, 0.5994, 0.1332, 0.7992, 0.5994, 0.1332, 0.7992,
        0.3996, 0.8658, 0.5994, 0.1332, 0.7992, 0.5994, 0.1332, 0.666 ,
        0.999 , 0.8658, 0.5994, 0.1332],
       [0.8658, 0.5994, 0.1332, 0.7992, 0.5994, 0.1332, 0.7992, 0.3996,
        0.8658, 0.5994, 0.1332, 0.7992, 0.5994, 0.1332, 0.666 , 0.999 ,
        0.8658, 0.5994, 0.1332, 0.7992]])

In [34]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
trainX.shape
#testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

(4009, 1, 20)

In [35]:
#le = preprocessing.LabelEncoder()
#YN=utils.column_or_1d(Ydataset, warn=True)
#YO=le.fit_transform(trainY)
#YO

maxTokens=13
EMBEDDING_DIM = 512


In [36]:
trainX[1:7,:]
print(trainY)

[15 13  9 ...  2  1  9]


In [37]:
trainYO = to_categorical(trainY, num_classes=Ysize)
trainYO

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [54]:
# create and fit the LSTM network
#https://github.com/ar-ms/lstm-mnist/blob/master/lstm_classifier.py
model = Sequential()
#model.add(ConvLSTM2D(32,(3,1) , activation='relu', input_shape=(None,None,look_back,1)))
model.add(LSTM(40,  input_shape=(1, look_back), return_sequences=True))
model.add(LSTM(16))
#model.add(Dense(30,activation="relu",input_dim=30))
model.add(Dropout(0.02))
#model.add(Dense(30,activation="relu"))
model.add(Dense(Ysize,activation="softmax"))
#model.add(Dense(32, input_shape=(20,)))
#model.add(Activation('relu'))
#model.add(Dense(25, activation='relu'))
#model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.compile(loss='categorical_crossentropy',      optimizer='adam',     metrics=['accuracy'])
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
unified_lstm_17 (UnifiedLSTM (None, 1, 40)             9760      
_________________________________________________________________
unified_lstm_18 (UnifiedLSTM (None, 16)                3648      
_________________________________________________________________
dropout_11 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 16)                272       
Total params: 13,680
Trainable params: 13,680
Non-trainable params: 0
_________________________________________________________________


In [56]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

### Train the residual compressor
earlystopper = EarlyStopping(patience=5, verbose=1) 
checkpointer = ModelCheckpoint('VSB_classifier', verbose=1, save_best_only=True)
#results = class_model.fit(X, y, validation_split=0.2, batch_size=50, epochs=300, 
#                    callbacks=[earlystopper, checkpointer])
#model.fit(trainX, trainYO, epochs=10, batch_size=1, verbose=2,validation_split=0.1,callbacks=[earlystopper, checkpointer])
model.fit(trainX,trainYO,epochs=70,validation_split=0.1)

Train on 3608 samples, validate on 401 samples
Epoch 1/70
3608/3608 [==============================] - 0s 50us/sample - loss: 0.5906 - accuracy: 0.8434 - val_loss: 0.0559 - val_accuracy: 1.0000
Epoch 2/70
3608/3608 [==============================] - 0s 51us/sample - loss: 0.5804 - accuracy: 0.8489 - val_loss: 0.0591 - val_accuracy: 1.0000
Epoch 3/70
3608/3608 [==============================] - 0s 51us/sample - loss: 0.5784 - accuracy: 0.8484 - val_loss: 0.0527 - val_accuracy: 1.0000
Epoch 4/70
3608/3608 [==============================] - 0s 51us/sample - loss: 0.5711 - accuracy: 0.8484 - val_loss: 0.0557 - val_accuracy: 1.0000
Epoch 5/70
3608/3608 [==============================] - 0s 51us/sample - loss: 0.5697 - accuracy: 0.8517 - val_loss: 0.0518 - val_accuracy: 1.0000
Epoch 6/70
3608/3608 [==============================] - 0s 51us/sample - loss: 0.5599 - accuracy: 0.8548 - val_loss: 0.0507 - val_accuracy: 1.0000
Epoch 7/70
3608/3608 [==============================] - 0s 50us/sample 

3608/3608 [==============================] - 0s 50us/sample - loss: 0.3947 - accuracy: 0.8961 - val_loss: 0.0252 - val_accuracy: 1.0000
Epoch 57/70
3608/3608 [==============================] - 0s 51us/sample - loss: 0.3930 - accuracy: 0.8925 - val_loss: 0.0252 - val_accuracy: 1.0000
Epoch 58/70
3608/3608 [==============================] - 0s 51us/sample - loss: 0.3888 - accuracy: 0.8911 - val_loss: 0.0259 - val_accuracy: 1.0000
Epoch 59/70
3608/3608 [==============================] - 0s 51us/sample - loss: 0.3843 - accuracy: 0.8947 - val_loss: 0.0250 - val_accuracy: 1.0000
Epoch 60/70
3608/3608 [==============================] - 0s 51us/sample - loss: 0.3837 - accuracy: 0.8927 - val_loss: 0.0206 - val_accuracy: 1.0000
Epoch 61/70
3608/3608 [==============================] - 0s 50us/sample - loss: 0.3788 - accuracy: 0.8952 - val_loss: 0.0244 - val_accuracy: 1.0000
Epoch 62/70
3608/3608 [==============================] - 0s 50us/sample - loss: 0.3787 - accuracy: 0.8963 - val_loss: 0.0223

In [53]:
trainYO.shape

(4009, 16)

In [ ]:
nn=model.predict(trainX[1:30])
print('ll')
nn2=numpy.argmax(nn,1)
print(nn2)
print(trainY[1:30])

In [ ]:
score = model.evaluate(trainX, trainY, verbose=1)

print('Test loss:', score)
#print('Test accuracy:', score)
